In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

df_courses = pd.read_csv("Coursera.csv")
df_courses

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,Ecole Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you will learn how to effective...,Data Analysis select (sql) database manageme...
...,...,...,...,...,...,...,...
3517,"Capstone: Retrieving, Processing, and Visualiz...",University of Michigan,Beginner,4.6,https://www.coursera.org/learn/python-data-vis...,"In the capstone, students will build a series ...",Databases syntax analysis web Data Visuali...
3518,Patrick Henry: Forgotten Founder,University of Virginia,Intermediate,4.9,https://www.coursera.org/learn/henry,"Give me liberty, or give me death: Remembering...",retirement Causality career history of the ...
3519,Business intelligence and data analytics: Gene...,Macquarie University,Advanced,4.6,https://www.coursera.org/learn/business-intell...,Megatrends heavily influence today's organisat...,analytics tableau software Business Intellig...
3520,Rigid Body Dynamics,Korea Advanced Institute of Science and Techno...,Beginner,4.6,https://www.coursera.org/learn/rigid-body-dyna...,"This course teaches dynamics, one of the basic...",Angular Mechanical Design fluid mechanics F...


In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

features = ['Course Name', 'University', 'Course Description', 'Skills']

df_courses['combined_features'] = df_courses[features].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf_vectorizer.fit_transform(df_courses['combined_features'])

knn_model = NearestNeighbors(n_neighbors=5, algorithm='brute', metric='cosine')
knn_model.fit(tfidf_matrix)

query_course_index = 10
query_course_tfidf = tfidf_matrix[query_course_index]
distances, indices = knn_model.kneighbors(query_course_tfidf, n_neighbors=5)

print("Recommended Courses:")
for i, index in enumerate(indices.flatten()):
    if index != query_course_index:  # exclude the query course itself
        print(f"{i + 1}. {df_courses.loc[index, 'Course Name']} - {df_courses.loc[index, 'University']}")

Recommended Courses:
2. Agile Projects: Defining Epics and Mapping Value with Taiga - Coursera Project Network
3. Agile Projects: Develop Product Wireframe Prototypes in Miro - Coursera Project Network
4. Agile Projects: Creating User Stories with Value in Taiga - Coursera Project Network
5. Product Development: Customer Persona Development with Miro - Coursera Project Network


In [10]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

def recommend_courses(query_course_index, courses_df, n_neighbors=5):
    features = ['Course Name', 'University', 'Course Description', 'Skills']
    courses_df['combined_features'] = courses_df[features].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(courses_df['combined_features'])

    knn_model = NearestNeighbors(n_neighbors=n_neighbors, algorithm='brute', metric='cosine')
    knn_model.fit(tfidf_matrix)

    query_course_tfidf = tfidf_matrix[query_course_index]
    distances, indices = knn_model.kneighbors(query_course_tfidf, n_neighbors=n_neighbors + 1)  # Include the query course itself

    recommendations_df = courses_df.loc[indices.flatten()]
    recommendations_df = recommendations_df[['Course Name', 'University', 'Difficulty Level', 'Course Description', 'Skills']]
    
    recommendations_df = recommendations_df.drop(index=query_course_index)
#     print("Recommended Courses:")
    return recommendations_df

recommendations = recommend_courses(10, df_courses)
recommendations

,Course Name,University,Difficulty Level,Course Description,Skills
3311,Agile Projects: Defining Epics and Mapping Val...,Coursera Project Network,Beginner,"By the end of this guided project, you will be...",Mapping Project Management presentation Pro...
2147,Agile Projects: Develop Product Wireframe Prot...,Coursera Project Network,Beginner,"By the end of this guided project, you will be...",agile management modeling software Product ...
422,Agile Projects: Creating User Stories with Val...,Coursera Project Network,Beginner,This guided project was developed to engage an...,Project Management Mapping Leadership and Ma...
3232,Product Development: Customer Persona Developm...,Coursera Project Network,Beginner,"By the end of this 2.5 project, you will be fl...",project modeling persona (user experience) ...
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,"By the end of this guided project, you will be...",Finance business plan persona (user experien...


In [15]:
print(tfidf_matrix)

  (0, 1518)	0.03345832593918906
  (0, 12463)	0.03816421030363664
  (0, 9055)	0.03438252527380232
  (0, 1549)	0.03419021365886642
  (0, 17018)	0.06024185830914848
  (0, 19979)	0.0571036548121268
  (0, 5380)	0.05377702505031194
  (0, 5768)	0.04462904356247318
  (0, 16663)	0.07048837659781934
  (0, 13797)	0.06024185830914848
  (0, 13475)	0.0571036548121268
  (0, 17420)	0.06735017310079766
  (0, 10077)	0.05547749862536456
  (0, 14974)	0.07048837659781934
  (0, 14311)	0.03765380304595764
  (0, 14722)	0.04958976382278587
  (0, 4469)	0.022900309140554707
  (0, 13346)	0.05041935771282351
  (0, 14970)	0.0462829994207339
  (0, 15665)	0.0464232544379296
  (0, 3940)	0.0526096555910639
  (0, 7678)	0.0650062179929324
  (0, 20137)	0.01929424616930855
  (0, 4221)	0.0453561739329616
  (0, 17736)	0.040617055896201903
  :	:
  (3521, 16922)	0.04086391710962664
  (3521, 2668)	0.04412233086754398
  (3521, 1343)	0.04216735695851023
  (3521, 16910)	0.04987446551430125
  (3521, 3901)	0.03024295924780012
  (352